# LCA of Germany's grid status quo

In [1]:
import bw2data as bd
import bw2calc as bc

/Users/timodiepers/anaconda3/envs/premise/lib/python3.11/site-packages/scikits/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


In [2]:
bd.projects.set_current("paper_plca_grid_expansion_rev")

In addition to the cumulated LCIA results, we want to investigate the contributions of certain grid components, materials and of the underlying directly emitting processes.

## Contributions of components and materials


Setup:

In [3]:
status_quo_node = bd.get_node(database="grid_status_quo", name="grid_status_quo")
fu = {status_quo_node: 1}

method = ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')

Now we calculate the LCI the standard way, but factorize the matrices because we will do additional calculations with the same matrices in the next step.

In [4]:
lca = bc.LCA(fu, method=method)
lca.lci(factorize=True)

/Users/timodiepers/anaconda3/envs/premise/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


Calculating the LCIA results for the component and material levels:

In [5]:
component_scores = {}
for component in status_quo_node.technosphere():
    material_scores = {}
    for material in component.input.technosphere():
        lca.lcia(demand={material.input.id: material.amount * component.amount})
        material_scores[material.input["name"]] = lca.score
    component_scores[component.input["name"]] = material_scores

Saving the results in a DataFrame and cleaning up the labels:

In [6]:
import pandas as pd

def map_labels(labels, groups, other_label="other"):
    new_labels = {}
    for label in labels:
        for key, group in groups.items():
            if key in label:
                new_labels[label] = group
                break
        else:
            new_labels[label] = other_label
    return new_labels

component_groups = {
    "Overhead line": "overhead lines",
    "cable": "cables",
    "Transformer": "transformers",
    "switchgear": "switchgears",
    "Substation": "substations",
}

material_groups = {
    "aluminium": "aluminum",
    "copper": "copper",
    "iron": "iron & steel",
    "steel": "iron & steel",
    "concrete": "concrete",
    "cement": "concrete",
    "sulfur hexafluoride": "SF6",
    "polyethylene": "plastics",
    "polypropylene": "plastics",
    "plastic": "plastics",
}

df = pd.DataFrame(component_scores)

df.index = df.index.map(
    map_labels(df.index, material_groups, other_label="other materials")
)
df.columns = df.columns.map(map_labels(df.columns, component_groups))

df_components_to_materials = df.groupby(level=0).sum().T.groupby(level=0).sum().T
df_components_to_materials


,cables,overhead lines,substations,switchgears,transformers
SF6,0.000000e+00,0.000000e+00,0.000000e+00,5.104115e+08,0.000000e+00
aluminum,1.472236e+10,1.656093e+10,0.000000e+00,3.429586e+08,7.705950e+08
concrete,0.000000e+00,2.643481e+09,1.983979e+08,0.000000e+00,0.000000e+00
copper,2.680742e+09,1.367915e+09,0.000000e+00,4.402503e+07,4.857178e+08
iron & steel,4.985923e+08,1.196019e+10,9.918893e+08,4.291867e+07,1.790604e+09
other materials,3.883319e+08,5.795345e+08,0.000000e+00,3.626520e+07,5.558916e+08
plastics,4.520817e+09,7.417500e+06,0.000000e+00,0.000000e+00,0.000000e+00


## Process contributions

To get the information which processes are the direct emitters, but keep the connection to the responsible materials, we use the initially created "aggregated material" activities that essentially sum up the demands for a certain material across all grid components.


First, we collect the aggregated materials nodes:

In [7]:
aggregated_material_names = [
    "aggregated material: aluminium",
    "aggregated material: steel",
    "aggregated material: concrete",
    "aggregated material: copper",
    "aggregated material: plastics",
    "aggregated material: other",
]

aggregated_material_nodes = [
    bd.get_node(name=name) for name in aggregated_material_names
]

Now, we do the individual LCAs, and aggregate the results by the reference product of the underlying directly emitting processes:

In [8]:
import bw2analyzer as ba

lca = bc.LCA(
    {aggregated_material_nodes[0]: 1}, method=method
)  # just to build the matrices
lca.lci(factorize=True)
material_top_processes = {}
for mat in aggregated_material_nodes:
    lca.lcia(demand={mat.id: 1})
    top_processes = ba.ContributionAnalysis().annotated_top_processes(lca, limit=5000)
    score_and_product = {}
    for process in top_processes:
        if process[2]["reference product"] in score_and_product:
            score_and_product[process[2]["reference product"]] += process[0]
        else:
            score_and_product[process[2]["reference product"]] = process[0]
    material_top_processes[mat["name"]] = score_and_product

/Users/timodiepers/anaconda3/envs/premise/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


We save the results in a DataFrame again and clean up the names:

In [9]:
product_groups = {
    "electricity": "electricity",
    "heat": "heat",
    "transport": "transport",
    "aluminium": "aluminum (process emissions)",
    "iron": "iron & steel (process emissions)",
    "steel": "iron & steel (process emissions)",
    "coal": "coal",
    "coke": "coal",
    "clinker": "clinker",
    "diesel": "transport",
    "Gas insulated switchgear": "SF6",
    "sulfur hexafluoride": "SF6",
}

df_aggregated_materials = pd.DataFrame(material_top_processes)

df_aggregated_materials.index = df_aggregated_materials.index.map(
    map_labels(
        df_aggregated_materials.index, product_groups, other_label="other processes"
    )
)
df_aggregated_materials.columns = df_aggregated_materials.columns.map(
    map_labels(
        df_aggregated_materials.columns, material_groups, other_label="other materials"
    )
)

df_materials_to_processes = (
    df_aggregated_materials.groupby(level=0).sum().T.groupby(level=0).sum().T
)
df_materials_to_processes

,aluminum,concrete,copper,iron & steel,other materials,plastics
SF6,1.296561e+06,1.262636e+04,2.217726e+05,6.131026e+04,4.897192e+08,5.564292e+04
aluminum (process emissions),4.644015e+09,5.191381e+06,2.365313e+07,6.308439e+06,3.653844e+06,6.891933e+06
clinker,7.282203e+07,1.896094e+09,1.535785e+08,2.286741e+08,2.659379e+07,2.783275e+07
coal,2.705392e+09,9.091097e+07,2.017607e+08,2.571040e+09,5.914327e+07,2.108768e+08
electricity,1.585490e+10,2.960198e+08,2.166626e+09,1.754544e+09,5.301698e+08,1.191607e+09
heat,5.498787e+09,5.878348e+07,4.418671e+08,4.079346e+08,2.659028e+08,1.615138e+08
iron & steel (process emissions),1.983241e+08,2.593146e+07,8.492762e+07,7.932718e+09,3.178363e+07,6.867541e+07
other processes,8.623850e+08,1.313899e+08,8.246568e+08,8.361027e+08,4.120756e+08,2.605654e+09
transport,2.558925e+09,3.316841e+08,6.811074e+08,1.546813e+09,2.572539e+08,2.551260e+08


In [10]:
product_groups = {
    "electricity": "electricity",
    "heat": "heat",
    "transport": "transport",
    "aluminium": "aluminum (process emissions)",
    "iron": "iron & steel (process emissions)",
    "steel": "iron & steel (process emissions)",
    "coal": "coal",
    "coke": "coal",
    "clinker": "clinker",
    "diesel": "transport",
    "Gas insulated switchgear": "SF6",
    "sulfur hexafluoride": "SF6",
}

df_aggregated_materials = pd.DataFrame(material_top_processes)

df_aggregated_materials.index = df_aggregated_materials.index.map(
    map_labels(
        df_aggregated_materials.index, product_groups, other_label="other processes"
    )
)
df_aggregated_materials.columns = df_aggregated_materials.columns.map(
    map_labels(
        df_aggregated_materials.columns, material_groups, other_label="other materials"
    )
)

df_materials_to_processes = (
    df_aggregated_materials.groupby(level=0).sum().T.groupby(level=0).sum().T
)
df_materials_to_processes

,aluminum,concrete,copper,iron & steel,other materials,plastics
SF6,1.296561e+06,1.262636e+04,2.217726e+05,6.131026e+04,4.897192e+08,5.564292e+04
aluminum (process emissions),4.644015e+09,5.191381e+06,2.365313e+07,6.308439e+06,3.653844e+06,6.891933e+06
clinker,7.282203e+07,1.896094e+09,1.535785e+08,2.286741e+08,2.659379e+07,2.783275e+07
coal,2.705392e+09,9.091097e+07,2.017607e+08,2.571040e+09,5.914327e+07,2.108768e+08
electricity,1.585490e+10,2.960198e+08,2.166626e+09,1.754544e+09,5.301698e+08,1.191607e+09
heat,5.498787e+09,5.878348e+07,4.418671e+08,4.079346e+08,2.659028e+08,1.615138e+08
iron & steel (process emissions),1.983241e+08,2.593146e+07,8.492762e+07,7.932718e+09,3.178363e+07,6.867541e+07
other processes,8.623850e+08,1.313899e+08,8.246568e+08,8.361027e+08,4.120756e+08,2.605654e+09
transport,2.558925e+09,3.316841e+08,6.811074e+08,1.546813e+09,2.572539e+08,2.551260e+08


## Prepare sankey diagram

Now we have all the information we need, we just need to get the data in the right format to feed our Sankey Diagram.


In [11]:
sankey_data = []  # will be filled with tuples of (source, target, value)

Grid to components:


In [12]:
for component, score in df_components_to_materials.sum().items():
    sankey_data.append((component, "grid status quo", score))

Components to materials:


In [13]:
for target, values in df_components_to_materials.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Materials to processes:


In [14]:
for target, values in df_materials_to_processes.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Create a DataFrame and save as .csv:

In [15]:
df_sankey = pd.DataFrame(sankey_data, columns=["source", "target", "value"])
df_sankey.value = df_sankey.value / 1e9
df_sankey.columns = ["source", "target", "value"]
df_sankey.to_csv("data/sankey_data.csv", index=False)

In [16]:
df_sankey

,source,target,value
0,cables,grid status quo,22.810845
1,overhead lines,grid status quo,33.119467
2,substations,grid status quo,1.190287
3,switchgears,grid status quo,0.976579
4,transformers,grid status quo,3.602809
...,...,...,...
76,electricity,plastics,1.191607
77,heat,plastics,0.161514
78,iron & steel (process emissions),plastics,0.068675
79,other processes,plastics,2.605654


The final figure for the paper was created using the Javascript library D3Blocks. The code is available on Observable: https://observablehq.com/@timo-diepers-ws/sankey-grid-status-quo-de. 